# Sky-MoCa -- Skyrmion Monte Carlo Code

Simulated Annealing for a 3D Spin Lattice

# Simulation

## Parameters & Initialization

In [ ]:
using HDF5
using LinearAlgebra
using ProgressMeter
using Random
using Statistics

### Type Definitions and Initialization

There's gonna be exactly one global instance of each of the types `Schedule`, `Parameters`, `Monitor`, and `Results`. For better performance all types are declared explicitly.

In [ ]:
mutable struct Schedule
    Ts::Vector{Float64} # the annealing schedule for the temperature
    Bs::Vector{Float64} # the annealing schedule for the external magnetic field
    Ntherm::Vector{Int64} # number of sweeps for thermalization before the stage
    Nsweep::Vector{Int64} # number of sweeps between two configurations at the stage
    Nconfig::Vector{Int64} # number of configurations at the stage
    N::Int64 # number of stages in the schedule
    os::Int64 # current stage index within the schedule
    function Schedule(Ts::Vector{Float64}, Bs::Vector{Float64}, Ntherm::Vector{Int64}, Nsweep::Vector{Int64}, Nconfig::Vector{Int64})
        N = length(Ts)+length(Bs)-1
        N == length(Nconfig) == length(Nsweep) || error("Invalid dimensions.")
        new(Ts, Bs, Ntherm, Nsweep, Nconfig, N, 1)
    end
    function Schedule(Ts::Vector{Float64}, Bs::Vector{Float64}; Ntherm::Int64=250, Nsweep::Int64=30, Nconfig::Int64=250)
        N = length(Ts)+length(Bs)-1
        Schedule(Ts, Bs, fill(Ntherm, N), fill(Nsweep, N), fill(Nconfig, N))
    end
    Schedule() = new([], [], [], [], [], 0, 0)
end

In [ ]:
mutable struct Parameters
    Nx::Int64 # number of lattice points in the x direction
    Ny::Int64 # number of lattice points in the y direction
    Nz::Int64 # number of lattice points in the z direction
    Ntot::Int64 # total number of lattice points
    T::Float64 # the current temperature
    J::Float64 # ferromagnetic exchange
    Jp::Float64 # ferromagnetic exchange of correction term
    K::Float64 # DM interaction
    Kp::Float64 # DM interaction of correction term
    B::Float64 # the current magnetic field in z direction
    denseoutput::Bool # switch whether to write out the configuration after each sweep
    openbnd::Bool # switch whether to open the boundaries in the z direction
    function Parameters(; Nx::Int64=30, Ny::Int64=30, Nz::Int64=30, T::Float64=0.0, J::Float64=1.0, K::Float64=tan(2pi/10.0), B::Float64=0.0, denseoutput::Bool=false, openbnd::Bool=false)
        Ntot = Nx*Ny*Nz; Jp = J/16.0; Kp = K/8.0
        new(Nx, Ny, Nz, Ntot, T, J, Jp, K, Kp, B, denseoutput, openbnd)
    end
    function Parameters(anneal::Schedule; Nx::Int64=30, Ny::Int64=30, Nz::Int64=30, denseoutput::Bool=false, openbnd::Bool=false)
        Parameters(Nx=Nx, Ny=Ny, Nz=Nz, T=anneal.Ts[1], B=anneal.Bs[1], denseoutput=denseoutput, openbnd=openbnd)
    end
end

In [ ]:
mutable struct Monitor
    Nrej::Int64 # number of overall rejected moves
    Nacc::Int64 # number of overall accepted moves
    nacc::Int64 # number of accepted moves per config
    accRate::Vector{Float64} # evolution of the acceptance rate over the configurations
    Monitor(anneal::Schedule) = new(0, 0, 0, zeros(Float64, sum(anneal.Nconfig)))
end

In [ ]:
mutable struct Results
    E::Vector{Float64} # history of total energy
    Etmp::Float64 # current total energy
    M::Array{Float64,2} # history of total magnetization
    Mtmp::Vector{Float64} # current total magnetization
    T::Vector{Float64} # history of temperature
    B::Vector{Float64} # history of external magnetic field
    os::Int64 # current offset (monte carlo time)
    function Results(anneal::Schedule)
        N = sum(anneal.Nconfig)
        new(zeros(Float64, N), 0.0, zeros(Float64, 3, N), zeros(Float64, 3), zeros(Float64, N), zeros(Float64, N), 0)
    end
    Results() = new([], 0, zeros(1, 1), [], [], [], 0)
end

In [ ]:
@inline init(anneal::Schedule) = (Monitor(anneal), Results(anneal))

### Grid Initialization

In [ ]:
function uniformS2(dims::Tuple{Int64, Int64, Int64})
    phi = 2pi * rand(dims...)
    z = 2 * rand(dims...) .- 1.0
    squ = sqrt.(1.0 .- z.*z)
    x = squ .* cos.(phi)
    y = squ .* sin.(phi)
    permutedims(cat(x, y, z, dims=4), [4, 1, 2, 3])
end

@inline function uniformS2!(s::Vector{Float64})
    phi = 2pi * rand()
    z = 2 * rand() - 1.0
    squ = sqrt(1.0 - z*z)
    s[1:3] = [squ*cos(phi), squ*sin(phi), z]
    nothing
end

In [ ]:
function initRandomGrid(dims::Tuple{Int64, Int64, Int64})
    grid = uniformS2(dims)
    res.Etmp = totalEnergy(grid)
    res.Mtmp = totalMagnetization(grid)
    @info "Initialized random grid."
    grid
end

@inline initRandomGrid() = initRandomGrid((pars.Nx, pars.Ny, pars.Nz))

In [ ]:
function initGridFromFile(fname::String)
    grid = Array(Float64, 3, pars.Nx, pars.Ny, pars.Nz)
    h5open(getPath(fname), "r") do file
        lastconfig = sort([parse(Int64, match(r"[0-9]+", dset).match) for dset in names(file["/configs"])])[end]
        grid = dropdims(file[string("/configs/configs_", lastconfig)][:,:,:,:,end], dims=5)
    end
    size(grid) == (3, pars.Nx, pars.Ny, pars.Nz) || error("Dimension mismatch.")
    res.Etmp = totalEnergy(grid)
    res.Mtmp = totalMagnetization(grid)
    grid
end

In [ ]:
@inline getX(grid::Array{Float64}) = grid[1, :]
@inline getY(grid::Array{Float64}) = grid[2, :]
@inline getZ(grid::Array{Float64}) = grid[3, :]
@inline getX(grid::Array{Float64,4}) = grid[1,:,:,:]
@inline getY(grid::Array{Float64,4}) = grid[2,:,:,:]
@inline getZ(grid::Array{Float64,4}) = grid[3,:,:,:]

In [ ]:
@inline idx(i::Int64, N::Int64) = mod(i-1, N) + 1

## Hamiltonian & Observables

In [ ]:
@inline crossX(a::Vector{Float64}, b::Vector{Float64}) = -a[3]*b[2] + a[2]*b[3]
@inline crossY(a::Vector{Float64}, b::Vector{Float64}) =  a[3]*b[1] - a[1]*b[3]
@inline crossZ(a::Vector{Float64}, b::Vector{Float64}) = -a[2]*b[1] + a[1]*b[2]

In [ ]:
function DMterm(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64, os::Int64)
    crossX(grid[:,i,j,k], grid[:,idx(i+os,pars.Nx),j,k]) +
    crossY(grid[:,i,j,k], grid[:,i,idx(j+os,pars.Ny),k]) +
    crossZ(grid[:,i,j,k], grid[:,i,j,idx(k+os,pars.Nz)])
end

In [ ]:
function totalEnergy(grid::Array{Float64,4})
    pars.openbnd::Bool ? totalEnergyOpen(grid) : totalEnergyPeriodic(grid)
end

function totalEnergyPeriodic(grid::Array{Float64,4})
    Nx = pars.Nx; Ny = pars.Ny; Nz = pars.Nz
    E = 0.0
    for k=1:Nz, j=1:Ny, i=1:Nx
        E += - pars.B * grid[3,i,j,k] -
        pars.J * dot(grid[:,i,j,k], grid[:,idx(i+1,Nx),j,k] + grid[:,i,idx(j+1,Ny),k] + grid[:,i,j,idx(k+1,Nz)]) +
        pars.Jp * dot(grid[:,i,j,k], grid[:,idx(i+2,Nx),j,k] + grid[:,i,idx(j+2,Ny),k] + grid[:,i,j,idx(k+2,Nz)]) -
        pars.K * DMterm(grid, i, j, k, 1) + pars.Kp * DMterm(grid, i, j, k, 2)
    end
    E
end

function totalEnergyOpen(grid::Array{Float64,4})
    Nx = pars.Nx; Ny = pars.Ny; Nz = pars.Nz
    E = 0.0
    for i=1:Nx
        for j=1:Ny
            for k=1:Nz-2
                E += -pars.B * grid[3,i,j,k] -
                pars.J * dot(grid[:,i,j,k], grid[:,idx(i+1,Nx),j,k] + grid[:,i,idx(j+1,Ny),k] + grid[:,i,j,idx(k+1,Nz)]) +
                pars.Jp * dot(grid[:,i,j,k], grid[:,idx(i+2,Nx),j,k] + grid[:,i,idx(j+2,Ny),k] + grid[:,i,j,idx(k+2,Nz)]) -
                pars.K * DMterm(grid, i, j, k, 1) + pars.Kp * DMterm(grid, i, j, k, 2)
            end
            E += -pars.J * dot(grid[:,i,j,Nz-1], grid[:,idx(i+1,Nx),j,Nz-1] + grid[:,i,idx(j+1,Ny),Nz-1] + grid[:,i,j,Nz]) +
            pars.Jp * dot(grid[:,i,j,Nz-1], grid[:,idx(i+2,Nx),j,Nz-1] + grid[:,i,idx(j+2,Ny),Nz-1]) -
            pars.K * DMterm(grid, i, j, Nz-1, 1) +
            pars.Kp * (crossX(grid[:,i,j,Nz-1], grid[:,idx(i+2,Nx),j,Nz-1]) + crossY(grid[:,i,j,Nz-1], grid[:,i,idx(j+2,Ny),Nz-1]))

            E += -pars.J * dot(grid[:,i,j,Nz], grid[:,idx(i+1,Nx),j,Nz] + grid[:,i,idx(j+1,Ny),Nz]) +
            pars.Jp * dot(grid[:,i,j,Nz], grid[:,idx(i+2,Nx),j,Nz] + grid[:,i,idx(j+2,Ny),Nz]) -
            pars.K * (crossX(grid[:,i,j,Nz], grid[:,idx(i+1,Nx),j,Nz]) + crossY(grid[:,i,j,Nz], grid[:,i,idx(j+1,Ny),Nz])) +
            pars.Kp * (crossX(grid[:,i,j,Nz], grid[:,idx(i+2,Nx),j,Nz]) + crossY(grid[:,i,j,Nz], grid[:,i,idx(j+2,Ny),Nz]))
        end
    end
    E
end

In [ ]:
function totalMagnetization(grid::Array{Float64,4})
    sum(grid, dims=(2,3,4))[:] / pars.Ntot
end

In [ ]:
@inline function energyChange(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64)
    pars.openbnd::Bool ? energyChangeOpen(grid, snew, i, j, k) : energyChangePeriodic(grid, snew, i, j, k)
end

function energyChangePeriodic(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64)
    Nx = pars.Nx::Int64
    Ny = pars.Ny::Int64
    Nz = pars.Nz::Int64
    im1 = idx(i-1,Nx); im2 = idx(i-2,Nx); ip1 = idx(i+1,Nx); ip2 = idx(i+2,Nx)
    jm1 = idx(j-1,Ny); jm2 = idx(j-2,Ny); jp1 = idx(j+1,Ny); jp2 = idx(j+2,Ny)
    km1 = idx(k-1,Nz); km2 = idx(k-2,Nz); kp1 = idx(k+1,Nz); kp2 = idx(k+2,Nz)
    sdiff = snew - grid[:,i,j,k]
    -pars.B::Float64 * sdiff[3] -
    pars.J::Float64  * dot(sdiff, grid[:,im1,j,k] + grid[:,ip1,j,k] + grid[:,i,jm1,k] + grid[:,i,jp1,k] + grid[:,i,j,km1] + grid[:,i,j,kp1]) +
    pars.Jp::Float64 * dot(sdiff, grid[:,im2,j,k] + grid[:,ip2,j,k] + grid[:,i,jm2,k] + grid[:,i,jp2,k] + grid[:,i,j,km2] + grid[:,i,j,kp2]) -
    pars.K::Float64  * (crossX(sdiff, grid[:,ip1,j,k] - grid[:,im1,j,k]) +
                        crossY(sdiff, grid[:,i,jp1,k] - grid[:,i,jm1,k]) +
                        crossZ(sdiff, grid[:,i,j,kp1] - grid[:,i,j,km1])) +
    pars.Kp::Float64 * (crossX(sdiff, grid[:,ip2,j,k] - grid[:,im2,j,k]) +
                        crossY(sdiff, grid[:,i,jp2,k] - grid[:,i,jm2,k]) +
                        crossZ(sdiff, grid[:,i,j,kp2] - grid[:,i,j,km2]))
end

function energyChangeOpen(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64)
    Nx = pars.Nx::Int64
    Ny = pars.Ny::Int64
    Nz = pars.Nz::Int64
    im1 = grid[:,idx(i-1,Nx),j,k]; im2 = grid[:,idx(i-2,Nx),j,k]
    ip1 = grid[:,idx(i+1,Nx),j,k]; ip2 = grid[:,idx(i+2,Nx),j,k]
    jm1 = grid[:,i,idx(j-1,Ny),k]; jm2 = grid[:,i,idx(j-2,Ny),k]
    jp1 = grid[:,i,idx(j+1,Ny),k]; jp2 = grid[:,i,idx(j+2,Ny),k]
    km1 = k == 1 ? zeros(3) : grid[:,i,j,idx(k-1,Nz)]; km2 = k <= 2 ? zeros(3) : grid[:,i,j,idx(k-2,Nz)]
    kp1 = k == Nz ? zeros(3) : grid[:,i,j,idx(k+1,Nz)]; kp2 = k >= Nz-1 ? zeros(3) : grid[:,i,j,idx(k+2,Nz)]
    sdiff = snew - grid[:,i,j,k]
    -pars.B::Float64 * sdiff[3] -
    pars.J::Float64  * dot(sdiff, im1 + ip1 + jm1 + jp1 + km1 + kp1) +
    pars.Jp::Float64 * dot(sdiff, im2 + ip2 + jm2 + jp2 + km2 + kp2) -
    pars.K::Float64  * (crossX(sdiff, ip1 - im1) + crossY(sdiff, jp1 - jm1) + crossZ(sdiff, kp1 - km1)) +
    pars.Kp::Float64 * (crossX(sdiff, ip2 - im2) + crossY(sdiff, jp2 - jm2) + crossZ(sdiff, kp2 - km2))
end

In [ ]:
@inline specificHeat(f::Vector{Float64}) = var(f) / (pars.Ntot * pars.T^2)

In [ ]:
@inline susceptibility(f::Vector{Float64}) = var(f) / (pars.Ntot * pars.T)

## Simulated Annealing

In [ ]:
function scheduleNext(anneal::Schedule)
    anneal.os += 1
    if anneal.os > length(anneal.Ts)
        pars.B = anneal.Bs[anneal.os - length(anneal.Ts)]
    else
        pars.T = anneal.Ts[anneal.os]
    end
end

In [ ]:
@inline acceptProb(del::Float64, T::Float64) = exp(-del/T)

In [ ]:
function acceptMove!(grid::Array{Float64,4}, snew::Vector{Float64}, dE::Float64, i::Int64, j::Int64, k::Int64)
    mon.Nacc::Int64 += 1
    mon.nacc::Int64 += 1
    res.Etmp::Float64 += dE
    res.Mtmp::Vector{Float64} += (snew - grid[:,i,j,k]) / pars.Ntot::Int64
    grid[:,i,j,k] = snew
end

In [ ]:
function update!(grid::Array{Float64,4}, snew::Vector{Float64})
    i,j,k = rand(1:pars.Nx::Int64, 3)
    uniformS2!(snew)
    dE = energyChange(grid, snew, i, j, k)
    if dE < 0.0 || acceptProb(dE, pars.T::Float64) >= rand()
        acceptMove!(grid, snew, dE, i, j, k)
    else
        mon.Nrej::Int64 += 1
    end
end

In [ ]:
@inline function sweep!(grid::Array{Float64,4}, snew::Vector{Float64})
    for i=1:pars.Ntot
        update!(grid, snew)
    end
end

In [ ]:
function nextconfig!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    for j=1:anneal.Nsweep[anneal.os]
        sweep!(grid, snew)
    end
end

In [ ]:
function thermalize!(grid::Array{Float64,4}, anneal::Schedule)
    @info "Start thermalizing..."
    snew = zeros(Float64, 3)
    @showprogress 1 "Thermalizing..." for i=1:anneal.Ntherm[anneal.os]::Int64
        sweep!(grid, snew)
    end
    @info "Finished thermalizing."
end

In [ ]:
function runstage!(grid::Array{Float64,4}, file::HDF5File, anneal::Schedule)
    pars.denseoutput && (conf = d_create(file, "configs/configs_$(anneal.os)", datatype(Float64), dataspace(3, pars.Nx, pars.Ny, pars.Nz, anneal.Nconfig[anneal.os]), "chunk", (3, pars.Nx, pars.Ny, pars.Nz, 1)))
    avg = d_create(file, "avgs/avg_$(anneal.os)", datatype(Float64), dataspace(3, pars.Nx, pars.Ny, pars.Nz))
    pars.denseoutput ? snapshot(grid, conf, avg, anneal) : snapshot(grid, avg, anneal)
    @showprogress 1 "Stage $(anneal.os)..." for i=1:anneal.Nconfig[anneal.os]::Int64-1
        nextconfig!(grid)
        pars.denseoutput ? snapshot(grid, conf, avg, anneal) : snapshot(grid, avg, anneal)
    end
    @info "Finished stage $(anneal.os)."
    @info "Write stage results..."
    stageOutput(file, avg, anneal)
    scheduleNext(anneal)
end

In [ ]:
function run(fname::String, anneal::Schedule; oldfile::String="")
    if oldfile == ""
        @info "Using new random grid."
        grid = initRandomGrid()
    else
        @info "Continue from last config of ", oldfile
        grid = initGridFromFile(oldfile)
    end
    run!(grid, fname, anneal)
end

function run!(grid::Array{Float64,4}, fname::String, anneal::Schedule)
    @info "Start run with $(anneal.N) stages..."
    thermalize!(grid, anneal)
    h5open(getPath(fname), "w") do file
        for i = 1:anneal.N::Int64
            runstage!(grid, file, anneal)
        end
        finalOutput(file, anneal)
    end
    @info "Finished all stages."
end

## Output

In [ ]:
@inline getPath(fname::String; ext::String="h5") = string("data/", fname, ".", ext)

In [ ]:
function snapshot(grid::Array{Float64,4}, avg::HDF5Dataset, anneal::Schedule)
    res.os += 1
    res.E[res.os] = res.Etmp
    res.M[:,res.os] = res.Mtmp
    res.T[res.os] = pars.T
    res.B[res.os] = pars.B
    mon.accRate[res.os] = mon.nacc / (pars.Ntot * anneal.Nsweep[anneal.os])
    mon.nacc = 0
    addConfig(grid, avg)
end

function snapshot(grid::Array{Float64,4}, conf::HDF5Dataset, avg::HDF5Dataset, anneal::Schedule)
    snapshot(grid, avg, anneal)
    appendConfig(grid, res.os-sum(anneal.Nconfig[1:anneal.os-1]), conf)
end

In [ ]:
function jackknife(obs::Function, f::Vector{Float64})
    N = length(f)
    jack = [obs([f[1:i]; f[i+2:N]]) for i=0:N-1]
    (N-1) * mean((jack .- mean(jack)).^2)
end

In [ ]:
function stageOutput(file::HDF5File, avg::HDF5Dataset, anneal::Schedule)
    averageConfigs!(avg, anneal)
    i = sum(anneal.Nconfig[1:anneal.os-1]) + 1
    f = i + anneal.Nconfig[anneal.os] - 1
    writeResultAndError(file, res.E[i:f], "res/E_avg_$(anneal.os)")
    writeResultAndError(file, getX(res.M[:,i:f]), "res/Mx_avg_$(anneal.os)")
    writeResultAndError(file, getY(res.M[:,i:f]), "res/My_avg_$(anneal.os)")
    writeResultAndError(file, getZ(res.M[:,i:f]), "res/Mz_avg_$(anneal.os)")
    writeResultAndError(file, res.E[i:f], "res/specific_heat_$(anneal.os)", specificHeat)
    writeResultAndError(file, getZ(res.M[:,i:f]), "res/susceptibility_$(anneal.os)", susceptibility)
end

In [ ]:
function finalOutput(file::HDF5File, anneal::Schedule)
    file["energy"] = res.E
    file["magnetization"] = res.M
    file["temperature"] = res.T
    file["magnetic_field"]=res.B
    file["acceptancerate"] = mon.accRate
    writeParameters(file, anneal)
end

In [ ]:
function writeParameters(file::HDF5File, anneal::Schedule)
    for structure in [pars, anneal], field in fieldnames(typeof(structure))
        try
            file[string("parameters/", string(field))] = getfield(structure, field)
        catch
            file[string("parameters/", string(field))] = Int64(getfield(structure, field))
        end
    end
end

In [ ]:
@inline function writeResultAndError(file::HDF5File, res::Float64, err::Float64, dset::String)
    file[dset] = [res, err]
end

function writeResultAndError(file::HDF5File, f::Vector{Float64}, dset::String, fun::Function=mean)
    res = fun(f)
    err = sqrt(jackknife(fun, f))
    writeResultAndError(file, res, err, dset)
end

In [ ]:
@inline function averageConfigs!(avg::HDF5Dataset, anneal::Schedule)
    avg[:,:,:,:] /= anneal.Nconfig[anneal.os]
end

In [ ]:
@inline function appendConfig(grid::Array{Float64,4}, os::Int64, conf::HDF5Dataset)
    conf[:,:,:,:,os] = grid
end

In [ ]:
@inline function addConfig(grid::Array{Float64,4}, avg::HDF5Dataset)
    avg[:,:,:,:] += grid
end

# Run the Simulation (scripting)

In [ ]:
# Set the random seed for reproducability
Random.seed!(42)
# Typical initialization. The arguments are as follows:
# First cool from T=1.5 to T=0.7 at fixed external field B=0.16 in steps of 0.1.
# Then decrease the external field from B=0.16 to B=0.0 in steps of 0.005.
# This gives a total of length(Ts)+length(Bs)-1 = 41 stages in the annealing schedule.
Ts = 1.5:-0.1:0.7
Bs = 0.16:-0.005:0.0
stages = length(Ts)+length(Bs)-1
# We thermalize just once in the very beginning for 10,000 sweeps.
therm = vcat(10000, zeros(Int64, stages-1))
# We take 30 sweeps between two neighboring configurations for all stages.
sweeps = fill(30, stages)
# We average over 2000 configurations during cooling and 1000 configurations afterwards.
configs = vcat(fill(2000, length(Ts)), fill(1000, length(Bs)-1))

# Setup the global parameters on a 30^3 grid with periodic boundary conditions.
anneal = Schedule(collect(Ts), collect(Bs), therm, sweeps, configs)
pars = Parameters(anneal, Nx=30, Ny=30, Nz=30, openbnd=false, denseoutput=false)
mon, res = init(anneal);

In [ ]:
# Start the run.
@time run("test", anneal);
# @time run("Ntot$(pars.Ntot)", anneal, oldfile="PathToFile/FileName");

# Postprocessing

## I/O

In [ ]:
function loadDataset(fname::String, dset::String, group=nothing)
    h5open(getPath(fname), "r") do file
        idx = group == nothing ? dset : string(group, "/", dset)
        read(file[idx])
    end
end

In [ ]:
function loadParameters!(fname::String, pars::Parameters, anneal::Schedule)
    h5open(getPath(fname), "r") do file
        for structure in [pars, anneal], field in fieldnames(typeof(structure))
            try
                setfield!(structure, field, read(file[string("parameters/", field)]))
            catch
                setfield!(structure, field, Bool(read(file[string("parameters/", field)])))
            end
        end            
    end
end

In [ ]:
function loadResults!(fname::String, res::Results)
    h5open(getPath(fname), "r") do file
        res.E = read(file[string("energy")])
        res.M = read(file[string("magnetization")])
        res.T = read(file[string("temperature")])
        res.B = read(file[string("magnetic_field")])
    end
end

In [ ]:
function exportPlotData(fname::String, data)
    fname = getPath(fname, ext="csv")
    mkpath(dirname(fname))
    @info fname
    writedlm(fname, data, ", ")
end

In [ ]:
# This writes out images of a plot that is built up datapoint by datapoint.
# It is useful to later create videos out of the images, where the plot is built up in real time.
function exportPlotImages(x::Array{Float64}, y::Array{Float64}, err::Array{Float64}, fname="img", ext="png", yl="y", xl="x")
    length(x) == length(y) == length(err) || error("Dimensions must agree.")
    for i = 1:length(x)
        I = 1:i
        errorbar(I, y[I], yerr=err[I], fmt=".")
        ylabel(yl)
        xlabel(xl)
        axis([min(x)/0.95, max(x)/0.95, min(y)/0.95, max(y)/0.95])
        savefig(string(fname, "_", lpad(i, 3, '0'), ".", ext))
    end
end

## Skyrmion Specific Analysis

In [ ]:
function drawProj(f::Array{Float64,3}, dir::Int64; name::String="proj", saveimg=false, ext="png")
    proj = dropdims(sum(f, dims=dir), dims=dir)
    img = imshow((proj/maximum(proj[:])))
    saveimg && img.write_png(getPath(string(name, "_", dir), ext=ext))
    img
end

function drawProj(f::Array{Float64,3}; name::String="proj", saveimg=false, ext="png")
    [drawProj(f, dir, name=name, saveimg=saveimg, ext=ext) for dir=1:3]
end

In [ ]:
@inline rotatehalf(N::Int64) = vcat(fld(N,2)+1:N, 1:fld(N,2))

In [ ]:
function braggIntensity(grid::Array{Float64,4})
    Sk = fftVF(grid)
    [norm(Sk[:,i,j,k])::Float64 for i=rotatehalf(size(Sk,2)), j=rotatehalf(size(Sk,3)), k=rotatehalf(size(Sk,4))]
end

In [ ]:
function fftVF(grid::Array{Float64,4})
    Sx = fft3(getX(grid))
    Sy = fft3(getY(grid))
    Sz = fft3(getZ(grid))
    permutedims(cat(Sx, Sy, Sz, dims=4), [4,1,2,3])
end

In [ ]:
function fft3(f::Array{Float64,3})
    FFTW.set_num_threads(4)
    fft(f .- mean(f[:]))
end

# Analyze the Results (scripting)

In [ ]:
using DelimitedFiles
using FFTW
using LsqFit
using PyPlot
using StatsBase

## Thermalization & Autocorrelation

Before any serious runs one has to check how fast the system thermalizes and by how many Monte Carlo steps configurations have to be separated to guarantee uncorrelated samples. To this end one should do a run at fixed temperature and external field, where one writes out the energy and magnetization at every single Monte Carlo step. Note that one might want to comment out the jackknife routine, because the double sum becomes very time consuming for a large number of configurations. One way to do this is:

In [ ]:
# Ts = [1.5]; Bs = [0.2]; therm = [0]; sweeps = [1]; configs = [10^6]
# anneal = Schedule(collect(Ts), collect(Bs), therm, sweeps, configs)
# pars = Parameters(anneal, Nx=10, Ny=10, Nz=10, openbnd=false, denseoutput=false)
# mon, res = init(anneal);

In [ ]:
# Load the necessary datasets (from a run like the one above).
name = "test"
pars = Parameters()
anneal = Schedule()
res = Results()
loadParameters!(name, pars, anneal)
loadResults!(name, res);

This is a sample analysis for a specific run. One has to play with the magic numbers in the rest of this section in order to analyze thermalization time and the (integrated) autocorrelation time. Once the magic numbers have been tweaked, you can set the `WRITEOUT` variable to `true` and reevaluate the cells to write out the data.

In [ ]:
WRITEOUT = true # global switch to suppress disk writes

In [ ]:
tMC = 1:sum(anneal.Nconfig)
# tMC = 1:6000 # show only beginning for better visibility
plot(res.E[tMC])
figure()
plot(tMC, getX(res.M)[tMC], tMC, getY(res.M)[tMC], tMC, getZ(res.M)[tMC])
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/therm_energy", res.E[tMC]);
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/therm_mx", getX(res.M)[tMC]);
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/therm_my", getY(res.M)[tMC]);
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/therm_mz", getZ(res.M)[tMC]);

In [ ]:
# Icomp = 5000:sum(anneal.Nconfig) # start after themalization determined from previous cell
Icomp = 1:sum(anneal.Nconfig)
Iauto = Icomp .- minimum(Icomp)
Re = autocor(res.E[Icomp], Iauto)
Rmx = autocor(getX(res.M)[Icomp], Iauto)
Rmy = autocor(getY(res.M)[Icomp], Iauto)
Rmz = autocor(getZ(res.M)[Icomp], Iauto);

In [ ]:
Ishow = collect(1:1500) # only show reliable beginning of autocorrelation function
Ishow = collect(1:500)
I = Ishow .- 1
plot(Ishow, Re[Ishow], Ishow, Rmx[Ishow], Ishow, Rmy[Ishow], Ishow, Rmz[Ishow])
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/auto_energy", Re[Ishow]);
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/auto_mx", Rmx[Ishow]);
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/auto_my", Rmy[Ishow]);
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/auto_mz", Rmz[Ishow]);

In [ ]:
Ishow = collect(1:80) # (integrated) autocorrelation for the energy
I = Ishow .- 1
model(x, p) = exp.(- x./p[1])
fit = curve_fit(model, I, Re[Ishow], [10.])
plot(I, model(I, fit.param), I, Re[Ishow])
@info "energy autocorrelation time: fit τ = $(fit.param[1]), integrated τ = $(0.5 + sum(abs.(Re[Ishow[2:end]])))"
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/autofit_energy", zip(Re[Ishow], model(I, fit.param)));

In [ ]:
Ishow = collect(1:500) # (integrated) autocorrelation for the magnetization
I = Ishow .- 1
model(x, p) = exp.(-x./p[1])
fitx = curve_fit(model, I, Rmx[Ishow], [10.])
fity = curve_fit(model, I, Rmy[Ishow], [10.])
fitz = curve_fit(model, I, Rmz[Ishow], [10.])
plot(I, model(I, fitx.param), I, Rmx[Ishow], I, model(I, fity.param), I, Rmy[Ishow], I, model(I, fitz.param), I, Rmz[Ishow])
@info "Mx autocorrelation time: fit τ = $(fitx.param[1]), integrated τ = $(0.5 + sum(abs.(Rmx[Ishow[2:end]])))"
@info "My autocorrelation time: fit τ = $(fity.param[1]), integrated τ = $(0.5 + sum(abs.(Rmy[Ishow[2:end]])))"
@info "Mz autocorrelation time: fit τ = $(fitz.param[1]), integrated τ = $(0.5 + sum(abs.(Rmz[Ishow[2:end]])))"
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/autofit_mx", zip(Rmx[Ishow], model(I,fitx.param)));
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/autofit_my", zip(Rmy[Ishow], model(I,fity.param)));
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/autofit_mz", zip(Rmz[Ishow], model(I,fitz.param)));

In [ ]:
plot(mon.accRate)
WRITEOUT && exportPlotData("T$(pars.T)_B$(pars.B)/accrate", mon.accRate);

## Bragg Intensity

In [ ]:
# Simple example of how to compute and display a Bragg intensity pattern.
avg = loadDataset("test", "avgs/avg_1");
drawProj(braggIntensity(avg), 1, saveimg=false)

In [ ]:
# Save all three Bragg intensity projections as `.png` images for the first NUMSTAGES stages of a run to disk.
# This can be used to write out many images to create videos.
name = "test"
NUMSTAGES = 5
for i=1:5
    avg = loadDataset(name, string("/avgs/avg_", i));
    drawProj(braggIntensity(avg), saveimg=true, name=string("imgbragg_", lpad(i, 3, '0'), "_"))
end

## Phase Diagram (deprecated!)

In [ ]:
function plotResultT(Ts, Bs, dset, savedata=false)
    data = [loadDataset("highres/T$(T)_B$(B)_cmpct", dset) for T in Ts, B in Bs]
    fig,ax = PyPlot.subplots()
    for (i,B) in enumerate(Bs)
        val = [data[j,i][1] for j in eachindex(Ts)]
        err = [data[j,i][2] for j in eachindex(Ts)]
        savedata && exportPlotData("B$(B)_$(dset)", zip(Ts[4:end], val[4:end] + (i-1) * 0.5, err[4:end]));
        ax[:errorbar](Ts[4:end], val[4:end] + (i-1) * 0.5, yerr=err[4:end], fmt=".", label="B=$(B)")
    end
    ax[:legend](loc="upper right")
end

function plotResultB(Ts, Bs, dset, savedata=false)
    data = [loadDataset("highres/T$(T)_B$(B)_cmpct", dset) for T in Ts, B in Bs]
    fig,ax = PyPlot.subplots()
    for (i,T) in enumerate(Ts)
        val = [data[i,j][1] for j in eachindex(Bs)]
        err = [data[i,j][2] for j in eachindex(Bs)]
        savedata && exportPlotData("T$(T)_$(dset)", zip(val, err));
        ax[:errorbar](Bs[:], val[:], yerr=err[:], fmt=".", label="T=$(T)")
    end
    ax[:legend](loc="upper right")
end